In [144]:
import numpy as np
import pandas as pd
from functools import reduce

In [145]:
def load_data(filename: str) -> dict[str, pd.DataFrame]:
    data = {}
    with pd.HDFStore(filename) as store:
        for k in store.keys():
            data[k] = store.get(k)
    return data

# Very ugly code to convert dataframes to Python objects,
# I haven't found a way to do it in a more concise way.
# 
# df.values can only be a numpy array, and since it's typed,
# all indices in the '/measurements' dataframe get converted to
# floating point numbers
def transform_data(data: dict[str, pd.DataFrame]) -> tuple[
        dict[int, np.ndarray],
        list[tuple[int, int, int, float]],
        dict[int, np.ndarray]
    ]:
    i_list = list(data['/measurements'].get('i').values.flatten())
    j_list = list(data['/measurements'].get('j').values.flatten())
    k_list = list(data['/measurements'].get('k').values.flatten())
    thetas = list(data['/measurements'].get('theta').values.flatten())
    measurements = [(i, j, k, theta) for i, j, k, theta in zip(i_list, j_list, k_list, thetas)]
    points = {k: data['/r'].loc[[k]].values.flatten() for k in data['/r'].index}
    corrections = {k: data['/dr'].loc[[k]].values.flatten() for k in data['/dr'].index}
    return (points, measurements, corrections)


In [146]:
data1 = load_data('localization_data_1.hdf')
data2 = load_data('localization_data_2.hdf')
print(reduce(lambda acc, s: f'{acc}\n{s}',
             map(lambda x: f"{'-'*40}<({x[0][1:]})\n{x[1]}\n{'-'*30}\n",
                 data1.items())))

----------------------------------------<(dr)
            dx       dy
point                  
1     -0.03927  0.00000
2      0.03927 -0.03927
3      0.00000  0.03927
------------------------------

----------------------------------------<(measurements)
             i  j  k     theta
measurement                   
1            1  2  3  0.706858
------------------------------

----------------------------------------<(r)
         x    y
point          
1     -1.0  0.0
2      0.0  1.0
3      1.0  0.0
------------------------------



So, the equation is
$$
    \theta_{ijk} = \arccos\frac
    {\left( \tilde{r}_3 - \tilde{r}_1 \right) \cdot \left( \tilde{r}_2 - \tilde{r}_1 \right)}
    {\left| \tilde{r}_3 - \tilde{r}_1 \right| \left| \tilde{r}_2 - \tilde{r}_1 \right|}.
$$
Let's rewrite it and try to extract $dr$'s out of it:
$$ \cos\theta_{ijk} = \frac{\tilde{P}}{\tilde{R}}, $$
$$
    \cos\theta_{ijk} - \frac{P}{R} = \Delta\left( \frac{P}{R} \right) =
    \frac{\Delta P}{R} - \frac{P \Delta R}{R^2}.
$$
Numerator:
$$
    \Delta P = \Delta\left( r_k - r_i \right) \cdot \left( r_j - ri \right) +
    \left( r_k - r_i \right) \cdot \Delta \left( r_j - ri \right) = 
    \left( r_j - ri \right) \cdot \left( dr_k - dr_i \right) + 
    \left( r_k - ri \right) \cdot \left( dr_j - dr_i \right).
$$
Denominator:
$$
    \Delta R = \Delta \left| r_k - r_i \right| \cdot \left| r_j - r_i \right| +
    \left| r_k - r_i \right| \cdot \Delta \left| r_j - r_i \right| =
    \rho_{ji} \Delta \left| r_k - r_i \right| +
    \rho_{ki} \Delta \left| r_j - r_i \right| =
$$
$$
    \frac{\rho_{ji}}{\rho_{ki}}\left[ 
        \left( x_k - x_i \right) \cdot \left( dx_k - dx_i \right) + 
        \left( y_k - y_i \right) \cdot \left( dy_k - dy_i \right)
    \right] + 
$$
$$ \frac{\rho_{ki}}{\rho_{ji}}\left[ 
        \left( x_j - x_i \right) \cdot \left( dx_j - dx_i \right) + 
        \left( y_j - y_i \right) \cdot \left( dy_j - dy_i \right)
    \right].
$$

Finally:
$$
    \cos\theta_{ijk} - \frac{P}{R} = 
    \left[ \frac{P}{R}\left( \frac{\xi_{ji}}{\rho_{ji}^2} + \frac{\xi_{ki}}{\rho_{ki}^2} \right) -
        \frac{\xi_{ki} + \xi_{ji}}{R} \right] \cdot dx_i +
    \left[ \frac{P}{R}\left( \frac{\eta_{ji}}{\rho_{ji}^2} + \frac{\eta_{ki}}{\rho_{ki}^2} \right) -
        \frac{\eta_{ki} + \eta_{ji}}{R} \right] \cdot dy_i +
$$
$$
    \left( \frac{\xi_{ki}}{R} - \frac{P\xi_{ji}}{R\rho_{ji}^2} \right) \cdot dx_j +
    \left( \frac{\eta_{ki}}{R} - \frac{P\eta_{ji}}{R\rho_{ji}^2} \right) \cdot dy_j +
    \left( \frac{\xi_{ji}}{R} - \frac{P\xi_{ki}}{R\rho_{ki}^2} \right) \cdot dx_k +
    \left( \frac{\eta_{ji}}{R} - \frac{P\eta_{ki}}{R\rho_{ki}^2} \right) \cdot dy_k
$$
where
$$
    \rho_{\alpha\beta} = \sqrt{\left( x_{\alpha} - x_{\beta} \right)^2
    + \left( y_{\alpha} - y_{\beta} \right)^2},
$$
$$ \xi_{\alpha\beta} = x_{\alpha} - x_{\beta}, $$
$$ \eta_{\alpha\beta} = y_{\alpha} - y_{\beta}, $$
$$ P = \left(r_k - r_i \right) \cdot \left( r_j - r_i \right), $$
$$ R = \rho_{ki} \rho_{ji}, $$

In [147]:
def compute_weights(r_i: np.ndarray, r_j: np.ndarray, r_k: np.ndarray) -> list[float]:
    """Outputs coefficients for x_i, y_i, x_j, y_j, x_k, y_k defined in the equation above"""
    if not (r_k.shape == r_j.shape == r_i.shape == (2,)):
        raise ValueError('All vectors must be 2-dimensional')
    rho_ki = np.linalg.norm(r_k - r_i)
    rho_ji = np.linalg.norm(r_j - r_i)
    xi_ki, eta_ki = r_k - r_i
    xi_ji, eta_ji = r_j - r_i
    P = np.dot(r_k - r_i, r_j - r_i)
    R = rho_ki * rho_ji
    return [(P * (xi_ji/(rho_ji*rho_ji) + xi_ki/(rho_ki*rho_ki)) - (xi_ki + xi_ji)) / R,
            (P * (eta_ji/(rho_ji*rho_ji) + eta_ki/(rho_ki*rho_ki)) - (eta_ki + eta_ji)) / R,
            (xi_ki - ( P*xi_ji / (rho_ji*rho_ji) )) / R,
            (eta_ki - ( P*eta_ji / (rho_ji*rho_ji) )) / R,
            (xi_ji - ( P*xi_ki / (rho_ki*rho_ki))) / R,
            (eta_ji - ( P*eta_ki / (rho_ki*rho_ki))) / R]


def compute_coef(
    points: dict[int, np.ndarray],
    measurement: tuple[int, int, int, float]
) -> float:
    i, j, k, theta = measurement
    r_i, r_j, r_k = points[i], points[j], points[k]
    rho_ki = np.linalg.norm(r_k - r_i)
    rho_ji = np.linalg.norm(r_j - r_i)
    P = np.dot(r_k - r_i, r_j - r_i)
    R = rho_ki * rho_ji
    return np.cos(theta) - P/R


def get_indices(i: int, j: int, k: int) -> list[int]:
    inds =  [2*i - 2, 2*i - 1,
            2*j - 2, 2*j - 1,
            2*k - 2, 2*k - 1]
    return inds


def compute_row(
    points: dict[int, np.ndarray],
    measurement: tuple[int, int, int, float]
) -> np.ndarray:
    i, j, k, _ = measurement
    r_i, r_j, r_k = points[i], points[j], points[k]
    weights = compute_weights(r_i, r_j, r_k)
    indices = get_indices(i, j, k)
    row = np.zeros(len(points) * 2)
    for i, c in zip(indices, weights):
        row[i] = c
    return row


def build_matrix(
    points: dict[int, np.ndarray],
    measurements: list[tuple[int, int, int, float]]
) -> np.ndarray:
    return np.array(list(map(lambda m : compute_row(points, m), measurements)))


def build_coefs(
    points: dict[int, np.ndarray],
    measurements: list[tuple[int, int, int, float]]
) -> np.ndarray:
    return np.array(list(map(lambda m: compute_coef(points, m), measurements)))

Calculate and apply correction to the given datasets:

In [148]:
points, measurements, corrections = transform_data(data1)
A = build_matrix(points, measurements)
print(f'matrix:\n{A}\n\n')
c = build_coefs(points, measurements)
print(f'coefficients:\n{c}\n\n')
x = np.linalg.pinv(A) @ c
print(f'corrections:\n{x.reshape([len(points), 2])}\n')
print(f'answers:\n{np.array(list(corrections.values()))}')

matrix:
[[-3.53553391e-01 -7.85046229e-17  3.53553391e-01 -3.53553391e-01
   0.00000000e+00  3.53553391e-01]]


coefficients:
[0.05329941]


corrections:
[[-3.76883741e-02 -8.36850014e-18]
 [ 3.76883741e-02 -3.76883741e-02]
 [ 0.00000000e+00  3.76883741e-02]]

answers:
[[-0.0392699  0.       ]
 [ 0.0392699 -0.0392699]
 [ 0.         0.0392699]]


Almost correct. Let's try the other dataset:

In [149]:
points, measurements, corrections = transform_data(data2)
A = build_matrix(points, measurements)
print(f'matrix:\n{A}\n\n')
c = build_coefs(points, measurements)
print(f'coefficients:\n{c}\n\n')
x = np.linalg.pinv(A) @ c

matrix:
[[ 1.15819146 -0.01798807 -0.97232383 -0.4117469   0.          0.
   0.          0.         -0.18586763  0.42973498  0.          0.        ]
 [ 0.          0.          0.0185836  -0.75167197 -0.58394212  0.84747616
   0.          0.          0.56535852 -0.0958042   0.          0.        ]
 [ 0.          0.          0.          0.         -0.12381428  0.02098125
   0.4018371   0.01961747 -0.27802282 -0.04059872  0.          0.        ]
 [ 0.          0.          0.01774021 -0.71755826 -0.40915046  0.56046893
   0.          0.          0.          0.          0.39141025  0.15708933]]


coefficients:
[ 0.00764516  0.19138891 -0.11059915  0.15058478]




In [150]:
print(f'corrections:\n{x.reshape([len(points), 2])}')
print(f'answers:\n{np.array(list(corrections.values()))}')

corrections:
[[-1.20206468e-02  1.86694740e-04]
 [ 1.28981102e-02 -1.09246158e-01]
 [ 5.74898934e-03  6.32548025e-02]
 [-1.99717879e-01 -9.75011754e-03]
 [ 1.07571003e-01  2.12318507e-02]
 [ 8.55204235e-02  3.43229274e-02]]
answers:
[[-0.0124687   0.00019365]
 [ 0.0131732  -0.104997  ]
 [-0.0193151   0.0708246 ]
 [-0.123992   -0.0060532 ]
 [ 0.0700385   0.0109086 ]
 [ 0.0725637   0.0291229 ]]


Not quite what we've expected... First 2 values are OK, but, for example, point 4 is a bit off.

But it still makes sense provided angles become closer to their measured values:

In [151]:
def calculate_theta(points, i, j, k):
    r_i, r_j, r_k = points[i], points[j], points[k]
    return np.arccos(np.dot(r_k - r_i, r_j - r_i) / 
                     (np.linalg.norm(r_k - r_i) * np.linalg.norm(r_j - r_i)))

points_corrected = (np.array(list(points.values())) + x.reshape([6,2]))
angles = list(map(lambda m: {
    'before': calculate_theta(points, m[0], m[1], m[2]),
    'after': calculate_theta(points_corrected,  m[0] - 1, m[1] - 1, m[2] - 1),
    'answer': m[3]
    }, measurements))
for a in angles:
    print(a)

{'before': 1.578445234689331, 'after': 1.5688240182667426, 'answer': 1.5708}
{'before': 1.7633770898844312, 'after': 1.570394236650075, 'answer': 1.5708}
{'before': 0.21664349754130663, 'after': 0.6916293941435528, 'answer': 0.523599}
{'before': 1.2138516825638075, 'after': 1.0549002711043738, 'answer': 1.0472}


Not perfect, but it definitely brings angle values closer to what they should be, but there are strange discrepancies between computed and expected $dr$ values.